# Intro to theano.scan
In order to implement recurrent networks, we will need to be able to apply certain operations repeatedly. This can be handled by a for loop generating a theano expression, but if we do not know in advance how long the loop will be, this is not possible. In these situations we can use theano.scan to create these loops.

In [27]:
# imports we will need
import numpy as np
import theano
import theano.tensor as T

## Some simple examples
We begin by some very simple examples that should be coded as functions implementing for-loops

### Cumulative sum of integers
Write a function that returns the sum of the first `n` integers. Add a keyword `return_full_sequence=False`, which, if set to `True`, returns the full calculation sequence.

In [8]:
def cumulative_sum_int(n, return_full_sequence=False):
    result = 0
    if return_full_sequence:
        seq = []
    for i in range(1, n + 1):
        result += i
        if return_full_sequence:
            seq.append(result)
    if return_full_sequence:
        return seq
    return result

In [3]:
cumulative_sum_int(9, return_full_sequence=True)

[1, 3, 6, 10, 15, 21, 28, 36, 45]

In [9]:
cumulative_sum_int(10, return_full_sequence=False)

55

### Cumulative sum of squares
Do the same thing for a cumulative sum of squares (you can copy and paste the function to write the cumulative sum of integers)

In [6]:
def cumulative_sum_squares(n, return_full_sequence=False):
    result = 0
    if return_full_sequence:
        seq = []
    for i in range(1, n + 1):
        result += i ** 2
        if return_full_sequence:
            seq.append(result)
    if return_full_sequence:
        return seq
    return result

In [10]:
cumulative_sum_squares(9, return_full_sequence=True)

[1, 5, 14, 30, 55, 91, 140, 204, 285]

In [12]:
cumulative_sum_squares(10, return_full_sequence=False)

385

### Fibonacci series
Now write a function calucating the nth fibonacci number, the zeroth and first being 0 and 1

In [15]:
def fib(n, return_full_sequence=False):
    if n == 0:
        if return_full_sequence:
            return [0]
        return 0
    if n == 1:
        if return_full_sequence:
            return [0, 1]
        return 1
    pre1, pre2 = 1, 0
    if return_full_sequence:
        seq = [0, 1]
    for _ in range(2, n):
        pre1, pre2 = pre1 + pre2, pre1
        if return_full_sequence:
            seq.append(pre1)
    if return_full_sequence:
        return seq
    return pre1
    
        

In [16]:
fib(10, return_full_sequence=True)

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

In [17]:
fib(11)

55

## Generalizing a little
All of the above can be written as simple operations done in a loop.
Code a function that can take a binary (two inputs) function of `new_value` and `current_value` and returns the increment. `new_value` should be read from a sequence. Add an argument `initial_value` to be able to commence the iterations. Output the full sequence.

In [29]:
def incrementer(func, sequence, initial_value):
    results = [initial_value]
    for item in sequence:
        results.append(func(item, results[-1]))
    return results[1:]
    

Write a function that adds two values in order to re-create the first example

In [31]:
def f_add(new_item, curr_value):
    return new_item + curr_value


Apply `incrementer` to `f_add` and a sequence of values from 0 to 9 with an initial value of 0

In [32]:
incrementer(f_add, np.arange(10), 0)

[0, 1, 3, 6, 10, 15, 21, 28, 36, 45]

Now write a `f_add_square` function and apply `incrementer` to it.

In [34]:
def f_add_square(new_item, curr_value):
    return new_item ** 2 + curr_value

In [35]:
incrementer(f_add_square, np.arange(10), 0.)

[0.0, 1.0, 5.0, 14.0, 30.0, 55.0, 91.0, 140.0, 204.0, 285.0]

## Generalizing a little more
If you have tried to implement the fibonacci example this way, you will have noticed that 